In [0]:
!pip3 install tensorflow-gpu==1.4.0
!apt install imagemagick

### Clone the repo

In [0]:
import os

os.chdir('/content/')

!if [ ! -e ~/work ]; then git clone https://github.com/ericpts/CycleGAN-TensorFlow ~/work; fi

os.chdir('/content/work')

! git fetch origin master
! git reset --hard origin/master

In [0]:
!./init.py

In [0]:
!python3 ./build_data.py --X_input_dir data/comp/trainA --X_output_file data/tfrecords/black.tfrecords --Y_input_dir data/comp/trainB --Y_output_file data/tfrecords/colored.tfrecords

## Install a Drive FUSE wrapper, authenticate, and mount a Drive filesystem

In [0]:
  # Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p ~/drive
!google-drive-ocamlfuse ~/drive
!mkdir -p /content/drive/checkpoints
!if [ -e /content/work/checkpoints ]; then unlink /content/work/checkpoints; fi
!ln -sf /content/drive/checkpoints /content/work/checkpoints

In [0]:
import subprocess
import os
datetime=subprocess.check_output(['ls', 'checkpoints']).decode('utf-8').strip()
os.environ['DATETIME']=datetime

In [0]:
!ls checkpoints/$DATETIME

## Train the network

In [0]:
!python3 train.py --X data/tfrecords/black.tfrecords --Y data/tfrecords/colored.tfrecords --load_model $DATETIME

## Generate a sample image

In [0]:
!python3 export_graph.py --checkpoint_dir checkpoints/$DATETIME \
                          --XtoY_model bw2color.pb \
                          --YtoX_model color2bw.pb \
                          --image_size 256

## Color a sample picture

In [0]:
!wget https://images.pexels.com/photos/285286/pexels-photo-285286.jpeg -O /tmp/in.jpg
  
!python3 inference.py --model pretrained/bw2color.pb \
                     --input /tmp/in.jpg \
                     --output ~/drive/output_sample.jpg \
                     --image_size 256